# WikiGen Agent Workflow Testing

This notebook provides an environment to test the WikiGen agent-based story processing workflow using the new **BaseAgent architecture**. Each agent can be tested individually with different LLM models to demonstrate the flexibility and modularity of the system.

## 🏗️ BaseAgent Architecture

All WikiGen agents now inherit from `BaseAgent`, providing:
- **Default Models**: Each agent has configurable default provider/model
- **Override Flexibility**: Can override provider/model per call when needed  
- **Centralized LLM Logic**: Common error handling and validation
- **Clean API**: Simplified method signatures with optional parameters

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

Each agent demonstrates the BaseAgent pattern with different default models to show architectural flexibility.

## ⚙️ Setup

Make sure the Portkey Gateway is running to use the LLM Service:

```bash
docker run -d \
  --name portkey-gateway \
  -p 8787:8787 \
  portkeyai/gateway:latest
```

The following cells will:
- use the Portkey Gateway to test the LLM Service
- initialize the LLM Service
- import the WikiGen workflow agents
- load a test story from the `tests/resources/pokemon_amber/story` directory


In [10]:
# Cell 1: Setup and Configuration
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import asyncio
import logging
from uuid import uuid4
from typing import Dict, Any, List, cast

# Set environment to skip database for testing
os.environ["SKIP_DATABASE"] = "true"
os.environ["PORTKEY_BASE_URL"] = "http://localhost:8787/v1"  # Default Portkey Gateway

# Add the backend/src directory to sys.path
notebook_dir = Path.cwd()
if (notebook_dir / 'src').is_dir() and (notebook_dir / 'pyproject.toml').is_file():
    # This means we're likely in the backend/ directory itself
    sys.path.insert(0, str(notebook_dir / 'src'))
elif (notebook_dir.parent / 'src').is_dir() and (notebook_dir.parent / 'pyproject.toml').is_file():
    # This means we're likely in the backend/notebooks/ directory
    sys.path.insert(0, str(notebook_dir.parent / 'src'))
else:
    print("Warning: Could not automatically add 'src/' to Python path.")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
# Reduce noise from third-party loggers
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('uvicorn.access').setLevel(logging.WARNING)

print("🚀 WikiGen Agent Workflow Testing Environment")
print("=" * 60)
print(f"Current working directory: {os.getcwd()}")
print(f"Database mode: {'IN-MEMORY' if os.environ.get('SKIP_DATABASE') == 'true' else 'SUPABASE'}")
print(f"Portkey Gateway: {os.environ.get('PORTKEY_BASE_URL', 'Not configured')}")
print("Autoreload enabled. Changes to .py files in src/ will be reloaded.")
print("\n💡 This notebook tests WikiGen agents with real LLM calls.")
print("   Each agent can use different models to demonstrate flexibility.")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🚀 WikiGen Agent Workflow Testing Environment
Current working directory: /home/jimnix/gitrepos/shuscribe/backend/notebooks
Database mode: IN-MEMORY
Portkey Gateway: http://localhost:8787/v1
Autoreload enabled. Changes to .py files in src/ will be reloaded.

💡 This notebook tests WikiGen agents with real LLM calls.
   Each agent can use different models to demonstrate flexibility.


In [11]:
# Cell 2: Import Modules
from src.config import settings
from src.services.llm.llm_service import LLMService
from src.database.repositories import get_user_repository, get_story_repository
from src.schemas.llm.models import LLMMessage, LLMResponse
from src.core.story_loading import StoryLoaderFactory
from dotenv import dotenv_values

# WikiGen agent imports
from src.agents.wikigen import (
    WikiGenOrchestrator,
    ArcSplitterAgent,
    WikiPlannerAgent,
    ArticleWriterAgent,
    GeneralSummarizerAgent,
    ChapterBacklinkerAgent
)

print("✅ Modules imported successfully.")

# Display current settings
print("\n--- Current Settings ---")
print(f"DEBUG: {settings.DEBUG}")
print(f"ENVIRONMENT: {settings.ENVIRONMENT}")
print(f"SKIP_DATABASE: {settings.SKIP_DATABASE}")
print(f"PORTKEY_BASE_URL: {settings.PORTKEY_BASE_URL}")
print("DATABASE_MODE: In-Memory (Supabase skipped)")
print("------------------------")

print("\n🔧 Import Summary:")
print("✅ LLMService - Ready for testing")
print("✅ WikiGen Agents - All 5 agents + orchestrator imported") 
print("✅ Story Loading - Test story will be loaded")
print("✅ Repository Factory - In-memory repositories for testing")
print("✅ Pydantic Models - Type-safe schemas")


✅ Modules imported successfully.

--- Current Settings ---
DEBUG: True
ENVIRONMENT: development
SKIP_DATABASE: True
PORTKEY_BASE_URL: http://localhost:8787/v1
DATABASE_MODE: In-Memory (Supabase skipped)
------------------------

🔧 Import Summary:
✅ LLMService - Ready for testing
✅ WikiGen Agents - All 5 agents + orchestrator imported
✅ Story Loading - Test story will be loaded
✅ Repository Factory - In-memory repositories for testing
✅ Pydantic Models - Type-safe schemas


In [12]:
# Cell 3: Initialize Services and Load Test Story

print("🚀 Initializing Services and Loading Test Story...")
print("=" * 60)

# Initialize repositories and LLM service
user_repo = get_user_repository()
story_repo = get_story_repository()
llm_service = LLMService(user_repository=user_repo)

print("✅ Services initialized successfully!")
print(f"📁 Repository types: {type(user_repo).__name__}, {type(story_repo).__name__}")

# Load API keys from .env
env_values = dotenv_values()
print("\n🔑 Loading API Keys from .env...")

# Check which providers have API keys available
available_providers = []
for provider in llm_service.get_all_llm_providers():
    provider_id = provider.provider_id
    api_key = env_values.get(f"{provider_id.upper()}_API_KEY")
    if api_key:
        available_providers.append({
            'id': provider_id,
            'name': provider.display_name,
            'api_key': api_key
        })
        print(f"✅ {provider.display_name}: API key found")
    else:
        print(f"⏭️  {provider.display_name}: No API key found")

print(f"\n📊 Available Providers: {len(available_providers)}")

# Load test story
print("\n📖 Loading Test Story...")
story_directory_path = Path("../tests/resources/pokemon_amber/story")

try:
    input_story = StoryLoaderFactory.load_story(story_directory_path)
    print(f"✅ Loaded: {input_story.metadata.title}")
    print(f"   Author: {input_story.metadata.author}")
    print(f"   Chapters: {input_story.total_chapters}")
    print(f"   Genres: {', '.join(input_story.metadata.genres)}")
    
    # Store in repository for testing
    fake_owner_id = uuid4()
    stored_story = await story_repo.store_input_story(input_story, fake_owner_id)
    print(f"   Stored with ID: {stored_story.id}")
    
    # Prepare story content for agent testing
    story_content = "\n\n".join([
        f"# {chapter.title}\n{chapter.content}" 
        for chapter in input_story.chapters
    ])
    
    print(f"\n📊 Test Data Summary:")
    print(f"   Total characters: {len(story_content):,}")
    print(f"   First chapter: {input_story.chapters[0].title}")
    print(f"   Last chapter: {input_story.chapters[-1].title}")
    
except Exception as e:
    print(f"❌ Error loading story: {e}")
    raise

print(f"\n🎯 Ready for WikiGen agent testing with {len(available_providers)} LLM providers!")


🚀 Initializing Services and Loading Test Story...
✅ Services initialized successfully!
📁 Repository types: InMemoryUserRepository, InMemoryStoryRepository

🔑 Loading API Keys from .env...
✅ OpenAI: API key found
✅ Anthropic: API key found
✅ Google: API key found

📊 Available Providers: 3

📖 Loading Test Story...
✅ Loaded: Pokemon: Ambertwo
   Author: ChronicImmortality
   Chapters: 17
   Genres: Drama, Action, Adventure, Fantasy
   Stored with ID: 6370ff07-cefb-4731-ab42-c64b8ffeb835

📊 Test Data Summary:
   Total characters: 273,664
   First chapter: [Chapter 1] Truck-kun Strikes Again
   Last chapter: [Chapter 17] The Eye of the Storm

🎯 Ready for WikiGen agent testing with 3 LLM providers!


# Run individual agents

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

## ArcSplitter Agent

In [9]:
# Test ArcSplitter Agent - Updated for BaseAgent Architecture

print("\n🧪 Testing ArcSplitter Agent")
print("=" * 60)
print("This agent analyzes story structure and determines optimal arc boundaries.")

# Choose the first available provider for arc splitting
if not available_providers:
    print("❌ No API keys available. Please add API keys to your .env file.")
else:
    provider = available_providers[0]  # Use first available provider
    
    print(f"🔧 Testing with: {provider['name']}")
    print(f"📊 Story: {input_story.metadata.title} ({len(input_story.chapters)} chapters)")
    
    # Initialize the agent with default provider/model (BaseAgent pattern)
    default_model = llm_service.get_default_test_model_name_for_provider(provider['id'])
    if not default_model:
        print(f"❌ No default model found for {provider['name']}")
        default_model = "gpt-4o-mini"  # Fallback
    
    arc_splitter = ArcSplitterAgent(
        llm_service=llm_service,
        default_provider=provider['id'],
        default_model=default_model
    )
    
    print(f"🎯 Agent defaults: {arc_splitter.default_provider} / {arc_splitter.default_model}")
    
    try:
        print("\n⚡ Running arc analysis...")
        
        # Test the arc splitting functionality with new BaseAgent API
        arc_result = await arc_splitter.analyze_story(
            story_title=input_story.metadata.title,
            story_content=story_content,
            total_chapters=len(input_story.chapters),
            user_id=fake_owner_id,
            api_key=provider['api_key'],  # Use API key instead of provider/model
            genre="Isekai Fantasy"
        )
        
        print(f"✅ ArcSplitterAgent completed successfully!")
        print(f"📋 Result type: {type(arc_result)}")
        print(f"📄 Result preview: {str(arc_result)[:300]}...")
        
        # Store result for later use
        arc_analysis = arc_result
        
    except NotImplementedError as e:
        print(f"⚠️  ArcSplitterAgent not yet implemented: {e}")
        print("   This is expected - the agent logic is not fully implemented yet.")
        arc_analysis = None
        
    except Exception as e:
        print(f"❌ ArcSplitterAgent failed: {e}")
        import traceback
        traceback.print_exc()
        arc_analysis = None



🧪 Testing ArcSplitter Agent
This agent analyzes story structure and determines optimal arc boundaries.
🔧 Using: OpenAI / gpt-4.1-mini
📊 Story: Pokemon: Ambertwo (17 chapters)


TypeError: super(type, obj): obj must be an instance or subtype of type

## WikiPlanner Agent

In [ ]:
# Test WikiPlanner Agent - Updated for BaseAgent Architecture

print("\n🧪 Testing WikiPlanner Agent")
print("=" * 60)
print("This agent plans wiki structure and article organization.")

if not available_providers:
    print("❌ No API keys available.")
else:
    # Use a different provider for planning to show flexibility
    provider = available_providers[1] if len(available_providers) > 1 else available_providers[0]
    
    print(f"🔧 Testing with: {provider['name']}")
    
    # Initialize the agent with default provider/model
    default_model = llm_service.get_default_test_model_name_for_provider(provider['id'])
    if not default_model:
        print(f"❌ No default model found for {provider['name']}")
        default_model = "gpt-4o-mini"  # Fallback
    
    wiki_planner = WikiPlannerAgent(
        llm_service=llm_service,
        default_provider=provider['id'],
        default_model=default_model
    )
    
    print(f"🎯 Agent defaults: {wiki_planner.default_provider} / {wiki_planner.default_model}")
    
    try:
        print("\n⚡ Running wiki planning...")
        
        # Test the wiki planning functionality
        wiki_plan = await wiki_planner.create_plan(
            arc_content=story_content[:10000],  # Use first 10k characters for testing
            story_metadata={
                "title": input_story.metadata.title,
                "author": input_story.metadata.author,
                "genres": input_story.metadata.genres
            },
            mode="fresh",
            user_id=fake_owner_id,
            api_key=provider['api_key']
        )
        
        print(f"✅ WikiPlannerAgent completed successfully!")
        print(f"📋 Result type: {type(wiki_plan)}")
        print(f"📄 Result preview: {str(wiki_plan)[:300]}...")
        
        # Store result for later use
        wiki_plan_result = wiki_plan
        
    except NotImplementedError as e:
        print(f"⚠️  WikiPlannerAgent not yet implemented: {e}")
        print("   This is expected - the agent logic is not fully implemented yet.")
        wiki_plan_result = None
        
    except Exception as e:
        print(f"❌ WikiPlannerAgent failed: {e}")
        import traceback
        traceback.print_exc()
        wiki_plan_result = None


## ArticleWriterAgent

# Test ArticleWriter Agent - Updated for BaseAgent Architecture

print("\n🧪 Testing ArticleWriter Agent")
print("=" * 60)
print("This agent generates actual wiki article content from planning output.")

if not available_providers:
    print("❌ No API keys available.")
else:
    # Use a different provider for article writing to show flexibility
    provider = available_providers[2] if len(available_providers) > 2 else available_providers[0]
    
    print(f"🔧 Testing with: {provider['name']}")
    
    # Initialize the agent with default provider/model
    default_model = llm_service.get_default_test_model_name_for_provider(provider['id'])
    if not default_model:
        print(f"❌ No default model found for {provider['name']}")
        default_model = "gpt-4o-mini"  # Fallback
    
    article_writer = ArticleWriterAgent(
        llm_service=llm_service,
        default_provider=provider['id'],
        default_model=default_model
    )
    
    print(f"🎯 Agent defaults: {article_writer.default_provider} / {article_writer.default_model}")
    
    try:
        print("\n⚡ Running article writing...")
        
        # Test the article writing functionality
        mock_wiki_plan = {"articles": ["Character: Ash", "Location: Pallet Town"]}  # Mock plan
        
        articles = await article_writer.write_articles(
            wiki_plan=mock_wiki_plan,
            arc_content=story_content[:5000],  # Use first 5k characters for testing  
            story_metadata={
                "title": input_story.metadata.title,
                "author": input_story.metadata.author,
                "genres": input_story.metadata.genres
            },
            user_id=fake_owner_id,
            api_key=provider['api_key'],
            include_web_search=False  # Skip web search for testing
        )
        
        print(f"✅ ArticleWriterAgent completed successfully!")
        print(f"📋 Result type: {type(articles)}")
        print(f"📄 Result preview: {str(articles)[:300]}...")
        
        # Store result for later use
        articles_result = articles
        
    except NotImplementedError as e:
        print(f"⚠️  ArticleWriterAgent not yet implemented: {e}")
        print("   This is expected - the agent logic is not fully implemented yet.")
        articles_result = None
        
    except Exception as e:
        print(f"❌ ArticleWriterAgent failed: {e}")
        import traceback
        traceback.print_exc()
        articles_result = None


## GeneralSummarizerAgent

In [ ]:
# Test GeneralSummarizer Agent - Updated for BaseAgent Architecture

print("\n🧪 Testing GeneralSummarizer Agent")
print("=" * 60)
print("This agent creates summaries of various content types.")

if not available_providers:
    print("❌ No API keys available.")
else:
    # Cycle through providers to demonstrate model flexibility
    provider = available_providers[0]  # Back to first provider
    
    print(f"🔧 Testing with: {provider['name']}")
    
    # Initialize the agent with default provider/model
    default_model = llm_service.get_default_test_model_name_for_provider(provider['id'])
    if not default_model:
        print(f"❌ No default model found for {provider['name']}")
        default_model = "gpt-4o-mini"  # Fallback
    
    summarizer = GeneralSummarizerAgent(
        llm_service=llm_service,
        default_provider=provider['id'],
        default_model=default_model
    )
    
    print(f"🎯 Agent defaults: {summarizer.default_provider} / {summarizer.default_model}")
    
    try:
        print("\n⚡ Running arc summarization...")
        
        # Test the summarization functionality
        summary = await summarizer.summarize_arc(
            arc_content=story_content[:8000],  # Use first 8k characters for testing
            arc_metadata={
                "title": "Arc 1: The Beginning",
                "chapters": [1, 2, 3, 4, 5]
            },
            summary_type="brief",
            user_id=fake_owner_id,
            api_key=provider['api_key']
        )
        
        print(f"✅ GeneralSummarizerAgent completed successfully!")
        print(f"📋 Result type: {type(summary)}")
        print(f"📄 Result preview: {str(summary)[:300]}...")
        
        # Store result for later use
        summary_result = summary
        
    except NotImplementedError as e:
        print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
        print("   This is expected - the agent logic is not fully implemented yet.")
        summary_result = None
        
    except Exception as e:
        print(f"❌ GeneralSummarizerAgent failed: {e}")
        import traceback
        traceback.print_exc()
        summary_result = None


## WikiGenOrchestrator

In [ ]:
# Test ChapterBacklinker Agent - Updated for BaseAgent Architecture

print("\n🧪 Testing ChapterBacklinker Agent")
print("=" * 60)
print("This agent creates bidirectional links between chapters and wiki articles.")

if not available_providers:
    print("❌ No API keys available.")
else:
    # Use the second provider if available
    provider = available_providers[1] if len(available_providers) > 1 else available_providers[0]
    
    print(f"🔧 Testing with: {provider['name']}")
    
    # Initialize the agent with default provider/model
    default_model = llm_service.get_default_test_model_name_for_provider(provider['id'])
    if not default_model:
        print(f"❌ No default model found for {provider['name']}")
        default_model = "gpt-4o-mini"  # Fallback
    
    backlinker = ChapterBacklinkerAgent(
        llm_service=llm_service,
        default_provider=provider['id'],
        default_model=default_model
    )
    
    print(f"🎯 Agent defaults: {backlinker.default_provider} / {backlinker.default_model}")
    
    try:
        print("\n⚡ Running chapter backlinking...")
        
        # Test the backlinking functionality with mock data
        mock_chapters = [
            {"title": "Chapter 1", "content": story_content[:2000]},
            {"title": "Chapter 2", "content": story_content[2000:4000]}
        ]
        mock_wiki_articles = [
            {"title": "Ash Ketchum", "type": "character"},
            {"title": "Pallet Town", "type": "location"}
        ]
        
        enhanced_chapters = await backlinker.create_chapter_links(
            chapters=mock_chapters,
            wiki_articles=mock_wiki_articles,
            current_arc_id=1,
            user_id=fake_owner_id,
            api_key=provider['api_key'],
            max_links_per_chapter=10
        )
        
        print(f"✅ ChapterBacklinkerAgent completed successfully!")
        print(f"📋 Result type: {type(enhanced_chapters)}")
        print(f"📄 Result preview: {str(enhanced_chapters)[:300]}...")
        
        # Store result for later use
        enhanced_chapters_result = enhanced_chapters
        
    except NotImplementedError as e:
        print(f"⚠️  ChapterBacklinkerAgent not yet implemented: {e}")
        print("   This is expected - the agent logic is not fully implemented yet.")
        enhanced_chapters_result = None
        
    except Exception as e:
        print(f"❌ ChapterBacklinkerAgent failed: {e}")
        import traceback
        traceback.print_exc()
        enhanced_chapters_result = None
